# Bar traction

Let's suppose that we have a bar with

* Lenght $L$
* Elasticity $E$
* Poisson $\nu$
* Diameter $d$

And it's tracted with a force $P$, as shown in the figure

With the Euler-Bernoulli Beam, we will have

$$
N(x) = P
$$
$$
V(x) = 0
$$
$$
M(x) = 0
$$

Our solution is only

$$
u(x) = \dfrac{P}{EA} \cdot \dfrac{x}{L} = \dfrac{4P}{\pi E d^2} \cdot \dfrac{x}{L}
$$

Now, let's go to the code

# Implementation

The initial values

In [1]:
import numpy as np
L = 1
E = 4/np.pi
nu = 0.0  # This value doesn't matter to the final answer, but it's needed
d = 2
P = 4

We create the material we will use

In [2]:
from material import Isotropic
steel = Isotropic(E=E, nu=nu)

We create our section

In [3]:
from section import Circle
circle = Circle(R=d/2, nu=nu)

We create the beam

In [4]:
from beam import EulerBernoulli
A = (0, 0, 0)
B = (L, 0, 0)
bar = EulerBernoulli(A, B)
bar.material = steel
bar.section = circle

We add the displacement vector ```U``` and the force ```F```

The values of ```U``` are

$$
\bold{U} =
\begin{bmatrix}
u_{1x} & u_{1y} & u_{1z} & \theta_{1x} & \theta_{1y} & \theta_{1z} \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
u_{ix} & u_{iy} & u_{iz} & \theta_{ix} & \theta_{iy} & \theta_{2z} \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
u_{nx} & u_{ny} & u_{nz} & \theta_{nx} & \theta_{ny} & \theta_{nz} \\
\end{bmatrix}
$$



And the values of ```F``` are

$$
\bold{F} =
\begin{bmatrix}
F_{1x} & F_{1y} & F_{1z} & M_{1x} & M_{1y} & M_{1z} \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
F_{ix} & F_{iy} & F_{iz} & M_{ix} & M_{iy} & M_{2z} \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
F_{nx} & F_{ny} & F_{nz} & M_{nx} & M_{ny} & M_{nz} \\
\end{bmatrix}
$$

So, ```U[i]``` means the displacement (and rotation) of the point ```i```, and the ```F[i]``` is the force applied in the node ```i```

In [5]:
n = 2  # number of points
U = np.empty((n, 6), dtype="object")
F = np.zeros((n, 6))

We add the boundary conditions. 

For that, we only know that at point $A$ (index ```0```), $u_{x}$ is fixed.

So, ```U``` is like

$$
\bold{U}
= \begin{bmatrix}
0 & u_{Ay} & u_{Az} & \theta_{Ax} & \theta_{Ay} & \theta_{Az} \\
u_{Bx} & u_{By} & u_{Bz} & \theta_{Bx} & \theta_{By} & \theta_{Bz} \\ 
\end{bmatrix}
$$

In [6]:
U[0, 0] = 0  # At point A, ux = 0
print("U = ")
print(U)

U = 
[[0 None None None None None]
 [None None None None None None]]


We add the force at the point $B$.

For that, we know that at point $B$ (index ```1```), $F_{x} = P$.

So, ```F``` is like

$$
\bold{F} =
\begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 0 \\
P & 0 & 0 & 0 & 0 & 0
\end{bmatrix}
$$

In [7]:
F[1, 0] = P  # At point B, Fx = P
print("F = ")
print(F)

F = 
[[0. 0. 0. 0. 0. 0.]
 [4. 0. 0. 0. 0. 0.]]


We get the stiffness matrix $K$.

The matrix is given by

$$
\bold{K}_{11} = 
\begin{bmatrix}
\frac{EA_x}{L} & 0 & 0 & 0 & 0 & 0 \\
0 & \frac{12EI_{z}}{L^3} & 0 & 0 & 0 & \frac{6EI_{z}}{L^2} \\
0 & 0 & \frac{12EI_{y}}{L^3} &  0 & \frac{-6EI_{y}}{L^2} \\
0 & 0 & 0 & \frac{GJ}{L} & 0 & 0 \\
0 & 0 & \frac{-6EI_{y}}{L^2} & 0 & \frac{4EI_{y}}{L} & 0 \\
0 & \frac{6EI_{z}}{L^2}& 0 & 0 & 0 & \frac{4EI_{z}}{L} 
\end{bmatrix}
$$

$$
\bold{K}_{12} = 
\begin{bmatrix}
\frac{-EA_x}{L} & 0 & 0 & 0 & 0 & 0 \\
0 & \frac{-12EI_{z}}{L^3} & 0 & 0 & 0 & \frac{6EI_{z}}{L^2} \\
0 & 0 & \frac{-12EI_{y}}{L^3} &  0 & \frac{-6EI_{y}}{L^2} \\
0 & 0 & 0 & \frac{-GJ}{L} & 0 & 0 \\
0 & 0 & \frac{6EI_{y}}{L^2} & 0 & \frac{2EI_{y}}{L} & 0 \\
0 & \frac{-6EI_{z}}{L^2}& 0 & 0 & 0 & \frac{2EI_{z}}{L} 
\end{bmatrix}
$$

$$
\bold{K}_{22} = 
\begin{bmatrix}
\frac{EA_x}{L} & 0 & 0 & 0 & 0 & 0 \\
0 & \frac{12EI_{z}}{L^3} & 0 & 0 & 0 & \frac{-6EI_{z}}{L^2} \\
0 & 0 & \frac{12EI_{y}}{L^3} &  0 & \frac{6EI_{y}}{L^2} \\
0 & 0 & 0 & \frac{GJ}{L} & 0 & 0 \\
0 & 0 & \frac{6EI_{y}}{L^2} & 0 & \frac{4EI_{y}}{L} & 0 \\
0 & \frac{-6EI_{z}}{L^2}& 0 & 0 & 0 & \frac{4EI_{z}}{L} 
\end{bmatrix}
$$

$$
\bold{K} = 
\begin{bmatrix}
\bold{K}_{11} & \bold{K}_{12} \\
\bold{K}_{12}^T & \bold{K}_{22}
\end{bmatrix}
$$

For exemple, with the values of

$$
L = 1; \ \ \ \ \ \ E = \dfrac{4}{\pi}; \ \ \ \ \ \ \nu = 0 ; \ \ \ \ \ \ d = 2
$$

We have the matrix


$$
K = 
\begin{bmatrix}
4 & & & & & & -4 & & & & & \\
& 12 & & & & 6 & & -12 &  & & & 6 \\
& & 12 & & -6 & & & & -12 & & -6 & \\
& & & 1 & & & & & & -1 & & \\
& & -6 & & 4 & & & & 6 & & 2 & \\
& 6 & & & & 4 & & -6 &  & & & 2 \\
-4 & & & & & & 4 & & & & & \\
& -12 & & & & -6 & & 12 &  & & & -6 \\
& & -12 & & 6 & & & & 12 & & 6 & \\
& & & -1 & & & & & & 1 & & \\
& & -6 & & 2 & & & & 6 & & 4 & \\
& 6 & & & & 2 & & -6 &  & & & 4 \\
\end{bmatrix}
$$

In [8]:
K = bar.stiffness_matrix()
print("K = ")
print(K.reshape(12, 12))

K = 
[[  4.   0.   0.   0.   0.   0.  -4.   0.   0.   0.   0.   0.]
 [  0.  12.   0.   0.   0.   6.   0. -12.   0.   0.   0.   6.]
 [  0.   0.  12.   0.  -6.   0.   0.   0. -12.   0.  -6.   0.]
 [  0.   0.   0.   1.   0.   0.   0.   0.   0.  -1.   0.   0.]
 [  0.   0.  -6.   0.   4.   0.   0.   0.   6.   0.   2.   0.]
 [  0.   6.   0.   0.   0.   4.   0.  -6.   0.   0.   0.   2.]
 [ -4.   0.   0.   0.   0.   0.   4.   0.   0.   0.   0.   0.]
 [  0. -12.   0.   0.   0.  -6.   0.  12.   0.   0.   0.  -6.]
 [  0.   0. -12.   0.   6.   0.   0.   0.  12.   0.   6.   0.]
 [  0.   0.   0.  -1.   0.   0.   0.   0.   0.   1.   0.   0.]
 [  0.   0.  -6.   0.   2.   0.   0.   0.   6.   0.   4.   0.]
 [  0.   6.   0.   0.   0.   2.   0.  -6.   0.   0.   0.   4.]]


Now we solve:

In [9]:
from solver import solve
U, F = solve(K, F, U)
print("U = ")
print(U)
print("F = ")
print(F)

U = 
[[0 0.0 -1.2711137632955748e-32 -5.036040116209269e-17
  -9.426707840301335e-17 1.729915820564182e-17]
 [1.0 -4.710001158988464e-17 1.6828776378656905e-16
  -5.0360401162092794e-17 -2.4230844917012476e-16 -1.1149918138541112e-16]]
F = 
[[-4.  0.  0.  0.  0.  0.]
 [ 4.  0.  0.  0.  0.  0.]]


The expected value of the equation 

$$
u(x) = \dfrac{4P}{\pi E d^2} \cdot \dfrac{x}{L}
$$

at $x = L$ and with the given data, gives us 

$$
u(L) = \dfrac{4 \cdot 4}{\pi \cdot \dfrac{4}{\pi} \cdot \left(2\right)^2} \cdot \dfrac{1}{1} = 1
$$

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=302ab669-ec31-4508-87f9-ef972d3ec2c5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>